# Full text search and keyword association analysis

全文檢索與關聯分析

# Load preprocessed news dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('PTT_news_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,2025-04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


# Filter data by searching keywords from "content" column

## "in" is very powerful

In [4]:
df.content[0]

'原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-04-15 14:19記者署名：劉忠勇原文內容：日媒報導，台積電（2330）即將敲定「面板級」先進封裝技術規格，預定最快2027年開始小量試產。據報導，台積電新封裝技術的第一代版本，將採用300x300 mm的方形基板，比先前試做的510×515 mm小得多，但相較於傳統的圓形晶圓可用面積更大。台積電為了嚴格控管品質，決定先採用略小的基板。為加快開發進度，台積電正在桃園興建試產線，目標是在2027年左右開始小量試產。。心得/評論：--南茂哥四年投資友達 資產快腰斬 大獲全勝啦！有夠慘的不少小白推文說很棒喔！真是害死不少人第一 群創沒技術第二 群創股本大 如真貢獻也佔比不到營收5%產能kobe:轉型半導體值得投資'

In [6]:
qk = '台積電'
text = df.content[0]
qk in text

True

In [7]:
qk = '居家自主健康管理'
text = df.content[0]
qk in text

False

In [8]:
qk = '顧立雄'
text = df.content[0]
qk in text

False

In [11]:
qk = '2330'
text = df.content[0]
qk in text

True

### all() any()

In [13]:
user_keywords = ['台積電','2330']
text = df.content[0]
all((qk in text) for qk in user_keywords)

True

In [10]:
user_keywords = ['肺炎疫情全球延燒','居家自主健康管理']
text = df.content[0]
any((qk in text) for qk in user_keywords)

False

In [11]:
user_keywords = ['烏克蘭戰爭','外交部']
text = df.content[0]
any((qk in text) for qk in user_keywords)

True

### Using apply() and lambda function

In [16]:
# Use apply() and lambda function
user_keywords = ['川普','命令']
df.content.apply(lambda text: all((qk in text) for qk in user_keywords))

0      False
1       True
2      False
3      False
4      False
       ...  
263    False
264    False
265    False
266    False
267    False
Name: content, Length: 268, dtype: bool

# Filter data using the following function

In [17]:
from datetime import datetime, timedelta

In [18]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: keywords 
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in user_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in user_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [19]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText_v0(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query

In [20]:
user_keywords = ['川普','外交部']
cond='and'
cate='全部'
weeks=2

df_query = filter_dataFrame_fullText(user_keywords, cond, cate,weeks)
df_query.shape

(1, 14)

In [21]:
user_keywords = ['川普']
cond='and'
cate='全部'
weeks=2

df_query = filter_dataFrame_fullText(user_keywords, cond, cate,weeks)
df_query.shape

(43, 14)

# Get news title, category, and link

In [22]:
user_keywords = ['川普','外交部']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)
len(df_query)

1

In [23]:
df_query

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
225,Military_04-16_13,2025-04-16,軍事,[情報] 烏軍官方殲敵統計&情報數則 (25/4/16),[統計部分]自22/2/24到25/4/16以來的俄軍損失統計人員：約936210名 (+1...,暫無,暫無,"[('烏克蘭', 7), ('俄軍', 5), ('俄羅斯', 5), ('系統', 4),...","['[', '統計', '部分', ']', '自', '22/2/24', '到', '2...","['統計', '俄軍', '統計', '人員', '定翼機', '旋翼機', '坦克', '...","[NerToken(word='俄軍', ner='ORG', idx=(25, 27)),...","[('[', 'PARENTHESISCATEGORY'), ('統計', 'Na'), (...",https://www.ptt.cc/bbs/Military/M.1744794058.A...,NaN


In [25]:
for i in range(min(3, len(df_query))):  # 最多印出3筆
    print(i)
    print(df_query.iloc[i]['category'])
    print(df_query.iloc[i]['title'])
    print(df_query.iloc[i]['link'])


0
軍事
[情報] 烏軍官方殲敵統計&情報數則 (25/4/16)
https://www.ptt.cc/bbs/Military/M.1744794058.A.BE5.html


In [26]:
df_query.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
225,Military_04-16_13,2025-04-16,軍事,[情報] 烏軍官方殲敵統計&情報數則 (25/4/16),[統計部分]自22/2/24到25/4/16以來的俄軍損失統計人員：約936210名 (+1...,暫無,暫無,"[('烏克蘭', 7), ('俄軍', 5), ('俄羅斯', 5), ('系統', 4),...","['[', '統計', '部分', ']', '自', '22/2/24', '到', '2...","['統計', '俄軍', '統計', '人員', '定翼機', '旋翼機', '坦克', '...","[NerToken(word='俄軍', ner='ORG', idx=(25, 27)),...","[('[', 'PARENTHESISCATEGORY'), ('統計', 'Na'), (...",https://www.ptt.cc/bbs/Military/M.1744794058.A...,NaN


## All-in-one function to return category, title, link, and photo_link

"photo_link" will be use in the next app

In [29]:
user_keywords = ['川普','中國']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [30]:
len(df_query)

25

In [31]:
# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k=5):
    items = []
    for i in range( len(df_query[0:k]) ): # show only 5 articles
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link='' # 若沒圖片，就設定為空字串，在前端網頁解讀json格式時才不會錯誤
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 

In [32]:
get_title_link_topk(df_query, 3)

[{'category': '學術股市',
  'title': 'Re: [新聞] 川普傳打算以關稅談判孤立中國 可能要各',
  'link': 'https://www.ptt.cc/bbs/Stock/M.1744794681.A.85D.html',
  'photo_link': ''},
 {'category': '學術股市',
  'title': '[新聞] 川普大讚「關稅政策」有效！\u3000發文激讚',
  'link': 'https://www.ptt.cc/bbs/Stock/M.1744794813.A.004.html',
  'photo_link': ''},
 {'category': '學術股市',
  'title': 'Re: [請益] 中國繼續擺爛是不是要害慘全世界？',
  'link': 'https://www.ptt.cc/bbs/Stock/M.1744795403.A.4DD.html',
  'photo_link': ''}]

### Some photo_links are "NaN". 

If the photo_link value is 'nan', it will be a problem when converted to json format on Django!

    "NaN" cannot be converted to a JSON string. 

In [33]:
df_query.iloc[0]['photo_link']

np.float64(nan)

### Test the photo_link is NaN or not and replace it with empty string
    How to do?
    You can test a variable is "NaN" or not by using pd, np or math.

In [34]:
import pandas as pd
import numpy as np
import math

# we can use pandas, numpy or math to check if photo_link is NaN
x = float("nan")

print(f"It's pd.isna  : {pd.isna(x)}")
print(f"It's np.isnan  : {np.isnan(x)}")
print(f"It's math.isnan : {math.isnan(x)}")

It's pd.isna  : True
It's np.isnan  : True
It's math.isnan : True


In [35]:
df_query.iloc[0]['photo_link']

np.float64(nan)

In [36]:
photo_link = df_query.iloc[0]['photo_link']
# if photo_link value is NaN, replace it with empty string 
if pd.isna(photo_link):
    photo_link='' # 

In [37]:
photo_link

''

# Find some related keywords

    Find related words from the top_key_freq column
    相關詞有哪一些? 找出各篇文章的topk關鍵詞?

## All-in-one function: Get related keywords

In [38]:
from collections import Counter
# 相關詞有哪一些?找出各篇文章的topk關鍵詞加以彙整計算
# 不能用 "get_related_keys"當函數名稱，因為這是Django系統用的名稱
def get_related_words(df_query):
    counter=Counter() # this counter is for all articles
    for idx in range(len(df_query)):
        pair_dict = dict(eval(df_query.iloc[idx].top_key_freq))
        counter += Counter(pair_dict)
    return counter.most_common(20) #return list format

In [39]:
# This version is for reference
def get_related_words_v0(df_query):
    all_pairs={}
    for idx in range(len(df_query)):
        row = df_query.iloc[idx].top_key_freq
        pairs = eval(row)
        for pair in pairs:
            w,f = pair
            if w in all_pairs:
                all_pairs[w]+= f
            else:
                all_pairs[w] = f

    counter = Counter(all_pairs)
    return counter.most_common(20) #return list format
    #return dict(counter.most_common(20)) #return dict format

In [40]:
user_keywords = ['川普','中國']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [41]:
df_query.shape

(25, 14)

In [42]:
get_related_words(df_query)

[('中國', 200),
 ('美國', 184),
 ('關稅', 97),
 ('川普', 77),
 ('經濟', 65),
 ('台灣', 48),
 ('政府', 40),
 ('國家', 37),
 ('貿易', 29),
 ('政策', 24),
 ('產業', 20),
 ('風險', 20),
 ('晶片', 19),
 ('市場', 19),
 ('原文', 17),
 ('歐盟', 17),
 ('世界', 16),
 ('總統', 14),
 ('商品', 14),
 ('歐洲', 14)]

In [37]:
# get_related_words_v1(df_query)

## A Step by step demonstration (do it yourself)

In [43]:
user_keywords = ['川普','中國']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [44]:
df_query.head(2)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
1,Stock_04-16_2,2025-04-16,學術股市,Re: [新聞] 川普傳打算以關稅談判孤立中國 可能要各,從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...,暫無,暫無,"[('中國', 36), ('美國', 30), ('國家', 21), ('技術', 13...","['從', '廣場', '協議', '看', '2018', '美中', '貿易戰 ', '...","['廣場', '協議', '美中', '貿易戰 ', '升級版', '關稅', '事件', ...","[NerToken(word='美國鋼鐵', ner='ORG', idx=(57, 61)...","[('從', 'P'), ('廣場', 'Nc'), ('協議', 'Na'), ('看',...",https://www.ptt.cc/bbs/Stock/M.1744794681.A.85...,NaN
2,Stock_04-16_3,2025-04-16,學術股市,[新聞] 川普大讚「關稅政策」有效！ 發文激讚,川普大讚「關稅政策」有效！ 發文激讚：美國通膨下降了ETTODAY2025年04月16日 1...,暫無,暫無,"[('關稅', 9), ('川普', 6), ('美國', 4), ('政策', 3), (...","['川普', '大讚', '「', '關稅', '政策', '」', '有效', '！', ...","['川普', '關稅', '政策', '美國', '通膨', '記者', '吳美依', '美...","[NerToken(word='川', ner='PERSON', idx=(0, 1)),...","[('川普', 'Nb'), ('大讚', 'VC'), ('「', 'PARENTHESI...",https://www.ptt.cc/bbs/Stock/M.1744794813.A.00...,NaN


In [45]:
df_query.top_key_freq

1      [('中國', 36), ('美國', 30), ('國家', 21), ('技術', 13...
2      [('關稅', 9), ('川普', 6), ('美國', 4), ('政策', 3), (...
4      [('中國', 9), ('經濟', 3), ('內需', 3), ('川普', 2), (...
5      [('美國', 3), ('中國', 3), ('台灣', 2), ('瑞昱', 2), (...
6      [('中國', 21), ('經濟', 11), ('美國', 9), ('企業', 6),...
7      [('中國', 5), ('原文', 3), ('美國', 2), ('台灣', 2), (...
18     [('中國', 21), ('關稅', 19), ('美國', 13), ('魷魚', 9)...
23     [('經濟', 32), ('美國', 19), ('風險', 18), ('川普', 14...
24     [('總量', 4), ('貨運', 2), ('數據', 2), ('公司', 2), (...
25     [('台灣', 8), ('中國', 7), ('美國', 5), ('關稅', 4), (...
27     [('中國', 21), ('歐洲', 13), ('關稅', 8), ('經濟', 6),...
28     [('中國', 15), ('美國', 10), ('台灣', 7), ('川爺', 5),...
29     [('關稅', 10), ('台積電', 8), ('時間', 7), ('川普', 6),...
31     [('中國', 8), ('商務部', 7), ('貿易', 7), ('李成鋼', 6),...
39     [('中國', 14), ('美國', 14), ('關稅', 6), ('川普', 6),...
40     [('預期', 5), ('中國', 5), ('訂單', 4), ('關稅', 4), (...
45     [('美國', 3), ('川普', 3), ('歐盟', 2), ('關稅', 2), (...
46     [('美國', 9), ('關稅', 8), (

### From the word_freq pairs, we sum frequency for each word

    How? 
    The best way is to use dict. Here is a simple example.

In [41]:
[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]

[('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]

In [42]:
'''
{'陳時中': 7,
 '解封': 5,
 '疫情': 10,
 '檢疫期': 3}

 (key, value) key can't be duplicated.鍵值不能重複，轉成dict時不處理重複的，會被丟棄
'''

dict([('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)])

{'陳時中': 7, '解封': 5, '疫情': 4, '檢疫期': 3}

In [43]:
#  (key, value) key can't be duplicated.鍵值不能重複，轉成dict時不處理重複的，會被丟棄
dict([('王金平', 2), ('吳敦義', 7), ('吳敦義', 5)])

{'王金平': 2, '吳敦義': 5}

In [44]:
c1 = Counter(dict([('陳時中', 7), ('解封', 5), ('疫情', 4), ('檢疫期', 3)]))

In [45]:
c1

Counter({'陳時中': 7, '解封': 5, '疫情': 4, '檢疫期': 3})

In [46]:
c2 = Counter(dict([('馬英九', 2), ('陳時中', 7), ('蔡英文', 20)]))

In [47]:
c2

Counter({'蔡英文': 20, '陳時中': 7, '馬英九': 2})

In [48]:
c1+c2

Counter({'蔡英文': 20, '陳時中': 14, '解封': 5, '疫情': 4, '檢疫期': 3, '馬英九': 2})

In [46]:
### Now we can start to cout frequency from our word_freq pairs
df_query.iloc[0].top_key_freq

"[('中國', 36), ('美國', 30), ('國家', 21), ('技術', 13), ('位置', 12), ('科技', 9), ('關稅', 8), ('產業', 8), ('晶片', 8), ('總統', 8), ('台灣', 8), ('程度', 8), ('世界', 7), ('政策', 7), ('經濟', 7), ('產品', 6), ('廣場', 5), ('政府', 5), ('傷害', 5), ('局勢', 5)]"

In [47]:
# Convert to dictionary
dict(eval(df_query.iloc[0].top_key_freq))

{'中國': 36,
 '美國': 30,
 '國家': 21,
 '技術': 13,
 '位置': 12,
 '科技': 9,
 '關稅': 8,
 '產業': 8,
 '晶片': 8,
 '總統': 8,
 '台灣': 8,
 '程度': 8,
 '世界': 7,
 '政策': 7,
 '經濟': 7,
 '產品': 6,
 '廣場': 5,
 '政府': 5,
 '傷害': 5,
 '局勢': 5}

In [48]:
c1 = Counter(dict(eval(df_query.iloc[0].top_key_freq)))
c1.most_common(10)

[('中國', 36),
 ('美國', 30),
 ('國家', 21),
 ('技術', 13),
 ('位置', 12),
 ('科技', 9),
 ('關稅', 8),
 ('產業', 8),
 ('晶片', 8),
 ('總統', 8)]

In [49]:
c2 = Counter(dict(eval(df_query.iloc[1].top_key_freq)))
c2.most_common(10)

[('關稅', 9),
 ('川普', 6),
 ('美國', 4),
 ('政策', 3),
 ('記者', 2),
 ('總統', 2),
 ('發文', 2),
 ('全球', 2),
 ('產業', 2),
 ('通膨', 1)]

In [50]:
counter = c1+c2
counter.most_common(10)

[('中國', 36),
 ('美國', 34),
 ('國家', 21),
 ('關稅', 17),
 ('技術', 13),
 ('位置', 12),
 ('產業', 10),
 ('總統', 10),
 ('政策', 10),
 ('科技', 9)]

In [54]:
# Advance operation for reference
# Using the itertools.groupby approach, you are summing the sorted groups based on first tuple elements.

# from itertools import groupby
# from operator import itemgetter

# my_list = [('a',2),('a',3),('b',3),('c',2),('b',4)]
# first = itemgetter(0)
# sums = [(k, sum(item[1] for item in tups_to_sum))
#         for k, tups_to_sum in groupby(sorted(my_list, key=first), key=first)]
# Outputs:

# [('a', 5), ('b', 7), ('c', 2)]

## Prepare wordcloud data

    textSizeMin = 10 # new_min
    textSizeMax = 80 # new_max
    新餅多大: 80-10 = 70

    {'text': '黃重凱', 'size': 24}   ==> 80
     {'text': '吳釗燮', 'size': 4}   ==> 10
     舊餅多大: 24-4 = 20

     (24-4) / (24-4)  * ( 80-10  )  + 10  ==> 80
     (13-4) / (24-4)  * ( 80-10  )  + 10  ==> 41
     ...
     ...
     (4-4) / (24-4)  * ( 80-10  )  + 10  ==> 10




## Min-Max Scaling

<img src="https://image.slidesharecdn.com/tcdsp17-featureengineering-extendedversion-170719211326/95/feature-engineering-getting-most-out-of-data-for-predictive-models-tdc-2017-30-638.jpg?cb=1500581962" width="550">

<img src="https://chrisalbon.com/images/machine_learning_flashcards/MinMax_Scaling_print.png" width="550">


<img src='https://www.researchgate.net/publication/282541174/figure/fig1/AS:307388692353061@1450298583749/Min-max-method-of-normalization.png' width="550">

<img src='https://t4tutorials.com/wp-content/uploads/2019/09/Min-Max-Normalization-Equation-Pythone-Matlab.png' width="550">

<img src='scale_formula.jpg' width="550">

In [51]:
# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_word_clouddata(df_query):

    # (1) Get wf_pairs by calling get_related_words().
    wf_pairs = get_related_words(df_query)
    
    # (2) cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    # Scaling frequency value into an interval of from 20 to 120.
    textSizeMin = 20 # 最小字
    textSizeMax = 120 # 最大字
    
    # if all frequences are the same, "divided by zero" exception will arise.
    # In the following case, exception will arise.
    # We must deal with this.
    # [('AA', 1), ('BB', 1), ('CC', 1)]
    # Instead of (max_-min_), we use len(wf_pairs) as divisor.
    # every word size is 1 / len(wf_pairs) 
    # 當每個字的頻率都一樣時，讓每個字的高度大小都一樣，分子是1，分母是字數==>均分
    
    # 排除分母為0的情況
    # 這裡的min_是最小值，max_是最大值，這兩個值是頻率的大小
    if (min_ != max_):
        max_min_range = max_ - min_

    else:
        max_min_range = len(wf_pairs) # 關鍵詞的數量: 20個
        min_ = min_ - 1 # every size is 1 / len(wf_pairs)
    
    # word cloud chart data using proportional scaling
    # 排除分母為0的情況
    clouddata = [{'text':w, 'size':int(textSizeMin + (f - min_)/max_min_range * (textSizeMax-textSizeMin))} for w, f in wf_pairs]

    # 可能分母為0的情況
    # clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))} for w, f in wf_pairs]

    return   wf_pairs, clouddata 

In [52]:
user_keywords = ['川普','關稅']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

get_related_word_clouddata(df_query)

([('中國', 173),
  ('美國', 161),
  ('關稅', 108),
  ('經濟', 65),
  ('川普', 59),
  ('台灣', 44),
  ('國家', 38),
  ('貿易', 29),
  ('政策', 26),
  ('烏克蘭', 25),
  ('歐洲', 24),
  ('市場', 22),
  ('政府', 21),
  ('產業', 19),
  ('晶片', 18),
  ('風險', 18),
  ('俄羅斯', 16),
  ('全球', 15),
  ('總統', 14),
  ('產品', 14)],
 [{'text': '中國', 'size': 120},
  {'text': '美國', 'size': 112},
  {'text': '關稅', 'size': 79},
  {'text': '經濟', 'size': 52},
  {'text': '川普', 'size': 48},
  {'text': '台灣', 'size': 38},
  {'text': '國家', 'size': 35},
  {'text': '貿易', 'size': 29},
  {'text': '政策', 'size': 27},
  {'text': '烏克蘭', 'size': 26},
  {'text': '歐洲', 'size': 26},
  {'text': '市場', 'size': 25},
  {'text': '政府', 'size': 24},
  {'text': '產業', 'size': 23},
  {'text': '晶片', 'size': 22},
  {'text': '風險', 'size': 22},
  {'text': '俄羅斯', 'size': 21},
  {'text': '全球', 'size': 20},
  {'text': '總統', 'size': 20},
  {'text': '產品', 'size': 20}])

# Find paragraphs containing the keywords. 

    There may be too many related paragraphs, so we display only some of them on our Django website
    一一比對文章段落，找出關鍵詞所在的段落

## All-in-one function: Find related paragraphs

In [53]:
import re

# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開 功能有限
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs


import re
# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, user_keywords, cond, k=10):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"  # 在每段落文字後面加一個句號。
            # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
            if cond == 'and':
                if all([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
            elif cond == 'or':
                if any([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
    return same_para[0:k]


In [54]:
user_keywords = ['川普','關稅','中國']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [55]:
len(df_query)

18

In [56]:
get_same_para(df_query, user_keywords, 'and', k=10)

['」認為，輝達之所以這麼做，是因為他在去年11月5日當選，並且實施了關稅，心得評論：對等關稅4月才實施（除中國加徵至145-245%，其他經濟體暫緩90天剩10%），墨加關稅4月2號才正式實施，三月有的關稅大概就鋼鋁關稅25%，現在就出來吹了，恐怕要看到真實影響數據還要再幾個月吧？--油價是OPEC+的關係吧（？）鮑爾就要軟著陸成功 半途殺出川普搞砸一切，要是川普不這樣搞，可能真的可以軟著陸。',
 '感覺上你在反串不過中國的確在擺爛，而且早在川普發動關稅戰前就在擺爛中國要經濟轉型、要促進內需這套說法從胡錦濤時代就一直講了講到現在中國的內需還是過低中國加入WTO快25年，幾乎把世界市場吃了一大半結果不僅沒有辦法培養出相應的內需市場反而養出了一個頭重腳輕的怪胎就算今天不是川普發難，遲早也會有其他的美國總統或歐盟政治領袖採取動作川普被人抨擊的並不是針對中國這點，而是手段粗暴欠缺考量，地圖砲朝盟友開習近平的確是在擺爛，比較跟中國差不多人均GDP的國家中國的社會保障、社會安全這塊也是倒數的那就演變成國富民窮的情況民間超額儲蓄 > 儲蓄投入房地產 > 房地產泡沫，資產損失 > 消費更加緊縮中國政府這個時候不僅沒有想辦法去降低人民負擔反而還要加緊集中權力、收攏資源備戰也就是習近平認為現在是東昇西降，是百年難得一遇的戰略機遇他要強化集權、重新集中資源去打一場自認的"國運之戰"美國無謀引戰在先，但影響更大的，恐怕會是上述中國的誤判如果關稅戰爛尾還好，經濟就動盪個小半年就是川普個人政治生涯終結，大家事後慢慢修補他造成的破壞如果關稅戰無意觸發了中美之間全面性的對抗，經濟世界大戰後果不堪設想所謂 "結束所有戰爭的戰爭" 或 "一戰定乾坤" 從來都是妄想這點不管是熱戰、冷戰、經濟戰、科技戰都一樣--。',
 '原文標題：「中國扛不住245％關稅」大量工廠放長假了！2000萬人慘失業原文連結：發布時間：2025-04-16記者署名：林瑩真原文內容：隨著美國總統川普再度祭出對中高額關稅，中國經濟面臨新一波衝擊。',
 '川普關稅重擊中國出口，中國企業停產裁員潮蔓延自美方啟動貿易戰以來，中國多數出口導向企業生產計畫紛紛停擺，許多工廠選擇提前放長假，長江三角洲與珠江三角洲地區更出現大規模裁員潮，失業農民工紛紛返鄉，被迫「提前過年」。',
 '美國總統川普本月9日授權對多國祭出的對等關稅措施暫停實

## A Step by step demonstration

## Step1: cut_paragraph() function

In [57]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,2025-04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


In [58]:
text = df.content[0]
text

'原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-04-15 14:19記者署名：劉忠勇原文內容：日媒報導，台積電（2330）即將敲定「面板級」先進封裝技術規格，預定最快2027年開始小量試產。據報導，台積電新封裝技術的第一代版本，將採用300x300 mm的方形基板，比先前試做的510×515 mm小得多，但相較於傳統的圓形晶圓可用面積更大。台積電為了嚴格控管品質，決定先採用略小的基板。為加快開發進度，台積電正在桃園興建試產線，目標是在2027年左右開始小量試產。。心得/評論：--南茂哥四年投資友達 資產快腰斬 大獲全勝啦！有夠慘的不少小白推文說很棒喔！真是害死不少人第一 群創沒技術第二 群創股本大 如真貢獻也佔比不到營收5%產能kobe:轉型半導體值得投資'

In [59]:
# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs


In [60]:
paragraphs = cut_paragraph(text)
paragraphs

['原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-04-15 14:19記者署名：劉忠勇原文內容：日媒報導，台積電（2330）即將敲定「面板級」先進封裝技術規格，預定最快2027年開始小量試產',
 '據報導，台積電新封裝技術的第一代版本，將採用300x300 mm的方形基板，比先前試做的510×515 mm小得多，但相較於傳統的圓形晶圓可用面積更大',
 '台積電為了嚴格控管品質，決定先採用略小的基板',
 '為加快開發進度，台積電正在桃園興建試產線，目標是在2027年左右開始小量試產',
 '心得/評論：--南茂哥四年投資友達 資產快腰斬 大獲全勝啦！有夠慘的不少小白推文說很棒喔！真是害死不少人第一 群創沒技術第二 群創股本大 如真貢獻也佔比不到營收5%產能kobe:轉型半導體值得投資']

In [112]:
text

'民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。'

In [113]:
text.split('。')


['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意',
 '']

In [114]:
paragraphs = text.split('。')
list(filter(None, paragraphs))


['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意']

In [115]:
re.split('[。！!？?]', text) # regular expression 正規式 正則式

['民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝',
 '外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日',
 '外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問',
 '根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜',
 '外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物',
 '送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾',
 '募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部',
 '外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力',
 '外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意',
 '']

### How to cut paragraph? 

    (1) Approach 1:
    string.split() (it does not support regex)

    (2)Approach 2:
    Use regular expression正規式 re.split() 
    re.split() works fine

Simpe example to demonstrate the usage of re.split()

In [69]:
# Use string split().  It does not support regex.
# Split stentence using delimiter or separator '。'

text = '這是第1句話。這是第2句話?這是第3句話。'
text.split('。')

['這是第1句話', '這是第2句話?這是第3句話', '']

In [70]:
# It doesn't work. string split() method does not support regex
text.split('[。?]')

['這是第1句話。這是第2句話?這是第3句話。']

In [71]:
import re

In [72]:
re.split('。', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話?這是--第3句話', '']

In [73]:
# 如果納入多個符號去切割，必須用regular expression
# Here, [abc] will match if the string you are trying to match contains any of the a, b or c . 
# You can also specify a range of characters using - inside square brackets. [a-e] is the same as [abcde] . [1-4] is the same as [1234] .


In [74]:
# separator:。 ?
re.split('[。?]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話', '這是--第3句話', '']

In [75]:
# "|" means or 可以加上｜去分隔開來，特別適用於當切割符號是由多個字組成時。
# separator:。 ?
re.split(r'[。|?]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這是第1句話', '這是，第2句話', '這是--第3句話', '']

In [76]:
# "|" means or
# separator:。 ?
re.split('[句話|是]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這', '第1', '', '。這', '，第2', '', '?這', '--第3', '', '。']

In [116]:
# "|" means or
# separator:。 ?
re.split('[句話 是]', "這是第1句話。這是，第2句話?這是--第3句話。")

['這', '第1', '', '。這', '，第2', '', '?這', '--第3', '', '。']

#### How to remove the empty elements?

In [77]:
# Do you notice the last element is an empty string?

In [78]:
result = re.split(r'[。?]', "這是第1句話。這是，第2句話?這是--第3句話。")

In [79]:
# Using Python filter function
filter(None, result)

In [80]:
list(filter(None, result))

['這是第1句話', '這是，第2句話', '這是--第3句話']

In [81]:
# An alternative way
[item for item in result if item]

['這是第1句話', '這是，第2句話', '這是--第3句話']

## Step 2: Find paragraphs containing keywords

### All-in-one function

In [61]:
import re
# Find out all paragraphs where multiple keywords occur.
def get_same_para(df_query, user_keywords, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"  # 在每段落文字後面加一個句號。
            # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
            if cond == 'and':
                if all([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
            elif cond == 'or':
                if any([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
    return same_para[0:k]


In [62]:
user_keywords = ['川普','關稅']
cond='and'
cate='全部'
weeks=2
df_query = filter_dataFrame_fullText(user_keywords, cond, cate, weeks)

In [63]:
len(df_query)

20

In [64]:
get_same_para(df_query, user_keywords, 'and', k=10)

['最後，回到我是投資人的位置，關注所有後續經證實的可能影響經濟格局變化的政策，是極需要的；例如前幾天美國海關CBP宣布部份產品及部份高科技產品豁免清單，但仍有變數（半導體關稅、芬太尼關稅）如果以台灣及其它國家來看，最大受益者目前是台灣（在豁免比例與產品範圍-高於其他國家，更在高附加價值的半導體與科技零組件領域，占據少數真正受惠的核心位置）因此關注後續1. 90天後（至7月9號）台灣的談判結果，2. 還有7/9後各國實際公佈談判條件，（跟其他國家影響比較，希望台灣不要比他國差或差太多）3 韓國不久前宣布200億鎂以上的晶片產業扶持計畫，（關注未來程度是否可能會影響台灣未來利益；4. 半導體關稅、芬太尼關稅後續5. 礦產關稅後續 會再如何影響國際局勢（川普已簽署一項命令，指示商務部調查所有「關鍵礦產」的進口情況，確認徵收關稅的可能性）。',
 '川普大讚「關稅政策」有效！\u3000發文激讚：美國通膨下降了ETTODAY2025年04月16日 15:55記者吳美依／綜合報導美國總統川普發文表示，美國的通貨膨脹已經下降了，這一切都要歸功於關稅政策，儘管最近的全球市場動盪，正是他頒布的「對等關稅」所引起。',
 '剛剛在自家社群媒體Truth Social發文表示，川普14日在白宮會見薩爾瓦多總統布格磊（Nayib Bukele）表示，根據最新消費者物價報告，，川普宣布鬆綁一些關稅措施之後，全球股市於14日回彈。',
 '」此外，輝達（Nvidia）宣布將首度在美國生產AI超級電腦，而川普也將這件事，視為關稅政策取得成功的證據，「這是你所聽過最重大的公告之一，因為正如你們知道的，輝達幾乎控制著整個產業，這是世上最重要的產業之一。',
 '」認為，輝達之所以這麼做，是因為他在去年11月5日當選，並且實施了關稅，心得評論：對等關稅4月才實施（除中國加徵至145-245%，其他經濟體暫緩90天剩10%），墨加關稅4月2號才正式實施，三月有的關稅大概就鋼鋁關稅25%，現在就出來吹了，恐怕要看到真實影響數據還要再幾個月吧？--油價是OPEC+的關係吧（？）鮑爾就要軟著陸成功 半途殺出川普搞砸一切，要是川普不這樣搞，可能真的可以軟著陸。',
 '感覺上你在反串不過中國的確在擺爛，而且早在川普發動關稅戰前就在擺爛中國要經濟轉型、要促進內需這套說法從胡錦濤時代就一直講了講到現在中國的內需

### Step by step demonstration

In [65]:
df_query.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
1,Stock_04-16_2,2025-04-16,學術股市,Re: [新聞] 川普傳打算以關稅談判孤立中國 可能要各,從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...,暫無,暫無,"[('中國', 36), ('美國', 30), ('國家', 21), ('技術', 13...","['從', '廣場', '協議', '看', '2018', '美中', '貿易戰 ', '...","['廣場', '協議', '美中', '貿易戰 ', '升級版', '關稅', '事件', ...","[NerToken(word='美國鋼鐵', ner='ORG', idx=(57, 61)...","[('從', 'P'), ('廣場', 'Nc'), ('協議', 'Na'), ('看',...",https://www.ptt.cc/bbs/Stock/M.1744794681.A.85...,NaN


In [66]:
text = df_query.content.iloc[0]
text

'從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元過強，當時的時代背景-美國鋼鐵重鎮跟底特律汽車王國-從曾經是世界第一的位置 被日本超越，美國大量進口日本鋼鐵及汽車，導致本土產業利潤被壓縮，造成大規模就業流失，美國主要針對貿易逆差最大的對象-日本，跟多國簽廣場協議，對日本的主要影響-日幣升值，影響對美國出口商品利潤下降，2018則是中國崛起，但是威脅到美國，中國不是合作者的態度跟做事方法，而是例如強迫去到中國投資的外國企業-技術轉移，或是偷取技術或資料，轉移或偷取後，過幾年就開設同類型的公司競爭，然後政府補貼，取得市佔後賣到包含轉移或偷取技術的國家及其它國家，-傾銷。中國以補貼再傾銷到其他國家的產業已不在少數，面板，太陽能，手機，電動車，化妝品，甚至成熟製程晶片，及其它產業，現在中國成熟製程晶片也已經快到可以傾銷的地步，所以之前聯電股價受影響吧我想，如果拋開政黨意識或人物爭議，假設當年李登輝總統沒有立下政策限制-台積電技術在外國設廠，必須保持落後台灣技術一代，以中國政府補貼產業的態度，現在的台積電會是什麼模樣？如果讓中國也掌握先進製程晶片技術，也政府補貼及大量製造，然後傾銷各國，台積電怎麼辦？現在還會是領先全球的技術位置嗎？然後中國現在即使在美國限制先進晶片製程及設備的情況下，照樣能透過新加坡走私AI晶片，到了新加坡成為走私最大國之一的程度，也能利用白手套公司，騙取讓台積電幫忙代工，因為維持現狀的話，中國離掌握最先進科技的進度越來越近；如果等到中國真的做到掌握先進製程晶片技術，當全世界所有最賺錢 最科技先進的產業在中國手中，從手機，電動車到電腦到先進製程的AI晶片然後到軍火，進而成為世界霸主，中國會怎麼做？會怎麼對待其它國家？所有其它國家裡面，尤其以我們台灣而言，是好還是不好？中國成為世界霸主，跟美國是世界霸主的差別是什麼？見微知著，看中國現在整體經濟跟人民幸福度，還有所有跟中國合作的國家例如一帶一路，經濟情況都沒有更好，所以如果中國在取得世界霸主地位之後，即使是站它隊的，我覺得頂多也只有少數人享有好處。而如果把美國跟中國擬人化，看它們的“人品”（我們如果不看誰比較好，只看誰比較不好）美國根據已知歷史，幫助過日韓還有中國及其它國家的經濟或教育或醫療，整體結果大都為正面效果，而中國的幫助，通常是幫到他國的基礎建設可以被中國租

In [67]:
paragraphs = cut_paragraph(text)
paragraphs

['從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元過強，當時的時代背景-美國鋼鐵重鎮跟底特律汽車王國-從曾經是世界第一的位置 被日本超越，美國大量進口日本鋼鐵及汽車，導致本土產業利潤被壓縮，造成大規模就業流失，美國主要針對貿易逆差最大的對象-日本，跟多國簽廣場協議，對日本的主要影響-日幣升值，影響對美國出口商品利潤下降，2018則是中國崛起，但是威脅到美國，中國不是合作者的態度跟做事方法，而是例如強迫去到中國投資的外國企業-技術轉移，或是偷取技術或資料，轉移或偷取後，過幾年就開設同類型的公司競爭，然後政府補貼，取得市佔後賣到包含轉移或偷取技術的國家及其它國家，-傾銷',
 '中國以補貼再傾銷到其他國家的產業已不在少數，面板，太陽能，手機，電動車，化妝品，甚至成熟製程晶片，及其它產業，現在中國成熟製程晶片也已經快到可以傾銷的地步，所以之前聯電股價受影響吧我想，如果拋開政黨意識或人物爭議，假設當年李登輝總統沒有立下政策限制-台積電技術在外國設廠，必須保持落後台灣技術一代，以中國政府補貼產業的態度，現在的台積電會是什麼模樣？如果讓中國也掌握先進製程晶片技術，也政府補貼及大量製造，然後傾銷各國，台積電怎麼辦？現在還會是領先全球的技術位置嗎？然後中國現在即使在美國限制先進晶片製程及設備的情況下，照樣能透過新加坡走私AI晶片，到了新加坡成為走私最大國之一的程度，也能利用白手套公司，騙取讓台積電幫忙代工，因為維持現狀的話，中國離掌握最先進科技的進度越來越近；如果等到中國真的做到掌握先進製程晶片技術，當全世界所有最賺錢 最科技先進的產業在中國手中，從手機，電動車到電腦到先進製程的AI晶片然後到軍火，進而成為世界霸主，中國會怎麼做？會怎麼對待其它國家？所有其它國家裡面，尤其以我們台灣而言，是好還是不好？中國成為世界霸主，跟美國是世界霸主的差別是什麼？見微知著，看中國現在整體經濟跟人民幸福度，還有所有跟中國合作的國家例如一帶一路，經濟情況都沒有更好，所以如果中國在取得世界霸主地位之後，即使是站它隊的，我覺得頂多也只有少數人享有好處',
 '而如果把美國跟中國擬人化，看它們的“人品”（我們如果不看誰比較好，只看誰比較不好）美國根據已知歷史，幫助過日韓還有中國及其它國家的經濟或教育或醫療，整體結果大都為正面效果，而中國的幫助，通常是幫到他國的基

In [68]:
# Find out all paragraphs where multiple keywords occur.
user_keywords = ['川普','關稅']
cond='and'
same_para=[] # 存放含有關鍵字的段落
for para in paragraphs:
    para += "。" # 在每段落文字後面加一個句號。
    # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
    if cond=='and': 
        if all([kw in para for kw in user_keywords]):
            same_para.append(para) # 符合條件的段落para保存起來
    elif cond=='or':
        if any([kw in para for kw in user_keywords]):
            same_para.append(para)  # 符合條件的段落para保存起來
same_para


['最後，回到我是投資人的位置，關注所有後續經證實的可能影響經濟格局變化的政策，是極需要的；例如前幾天美國海關CBP宣布部份產品及部份高科技產品豁免清單，但仍有變數（半導體關稅、芬太尼關稅）如果以台灣及其它國家來看，最大受益者目前是台灣（在豁免比例與產品範圍-高於其他國家，更在高附加價值的半導體與科技零組件領域，占據少數真正受惠的核心位置）因此關注後續1. 90天後（至7月9號）台灣的談判結果，2. 還有7/9後各國實際公佈談判條件，（跟其他國家影響比較，希望台灣不要比他國差或差太多）3 韓國不久前宣布200億鎂以上的晶片產業扶持計畫，（關注未來程度是否可能會影響台灣未來利益；4. 半導體關稅、芬太尼關稅後續5. 礦產關稅後續 會再如何影響國際局勢（川普已簽署一項命令，指示商務部調查所有「關鍵礦產」的進口情況，確認徵收關稅的可能性）。']

### multiple words in text (easier way)

In [90]:
para = '民進黨重用派系、酬庸人事的部分，他不再贅述；不過，他也批評國民黨，前總統馬英九的「交通幫」，沒有解決桃園機場跑道和漏水的問題，馬政府有許多人害怕跟宋楚瑜多接觸，擔心因此被老闆換掉，同樣是小氣、沒有肚量。'

In [91]:
user_keywords = ['馬英九','宋楚瑜']

In [92]:
any([kw in para for kw in user_keywords])


True

In [93]:
all([kw in para for kw in user_keywords])


True

In [94]:
user_keywords = ['馬英九','蔡英文']

In [95]:
any([kw in para for kw in user_keywords])


True

In [96]:
all([kw in para for kw in user_keywords])


False

### re.search(): An alternative way

In [69]:
# An alternative way for advanced users: using re.seach()
# Alternative approach using re.search() for reference
user_keywords = ['川普','關稅']
cond='and'
same_para=[] # 存放含有關鍵字的段落
for para in paragraphs:
    para += "。"
    if cond=='and':
        if all([re.search(kw, para) for kw in user_keywords]):
            same_para.append(para)
    elif cond=='or':
        if any([re.search(kw, para) for kw in user_keywords]):
            same_para.append(para)
same_para


['最後，回到我是投資人的位置，關注所有後續經證實的可能影響經濟格局變化的政策，是極需要的；例如前幾天美國海關CBP宣布部份產品及部份高科技產品豁免清單，但仍有變數（半導體關稅、芬太尼關稅）如果以台灣及其它國家來看，最大受益者目前是台灣（在豁免比例與產品範圍-高於其他國家，更在高附加價值的半導體與科技零組件領域，占據少數真正受惠的核心位置）因此關注後續1. 90天後（至7月9號）台灣的談判結果，2. 還有7/9後各國實際公佈談判條件，（跟其他國家影響比較，希望台灣不要比他國差或差太多）3 韓國不久前宣布200億鎂以上的晶片產業扶持計畫，（關注未來程度是否可能會影響台灣未來利益；4. 半導體關稅、芬太尼關稅後續5. 礦產關稅後續 會再如何影響國際局勢（川普已簽署一項命令，指示商務部調查所有「關鍵礦產」的進口情況，確認徵收關稅的可能性）。']

In [98]:
text = '民進黨重用派系、酬庸人事的部分，他不再贅述；不過，他也批評國民黨，前總統馬英九的「交通幫」，沒有解決桃園機場跑道和漏水的問題，馬政府有許多人害怕跟宋楚瑜多接觸，擔心因此被老闆換掉，同樣是小氣、沒有肚量。'

In [99]:
key = ['馬英九','蔡英文']

In [100]:
any([re.search(kw, text) for kw in user_keywords])

False

In [101]:
all([re.search(kw, text) for kw in user_keywords])

False

# views.py in Django website

To save memory, we just import df from the other app as follows.
from app_user_keyword.views import df

In [ ]:
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
from django.http import JsonResponse

from datetime import datetime, timedelta
import pandas as pd
import math
import re
from collections import Counter

# (1) we can load data using read_csv() 自己app的csv檔案
# global variable
# df = pd.read_csv('dataset/cna_news_200_preprocessed.csv', sep='|')


# (2) we can load data using reload_df_data() function 隔壁app的csv檔案
# global variable
def load_df_data_v1():
    # global variable
    global  df
    df = pd.read_csv('app_user_keyword/dataset/cna_news_200_preprocessed.csv', sep='|')

# (3) df can be import from app_user_keyword 隔壁app的變數
# To save memory, we just import df from the other app as follows.
# from app_user_keyword.views import df

# (4) df can be import from app_user_keyword  隔壁app的變數
import app_user_keyword.views as userkeyword_views
def load_df_data():
    # import and use df from app_user_keyword 
    global df # global variable
    df = userkeyword_views.df

load_df_data()


# For the key association analysis
def home(request):
    return render(request, 'app_user_keyword_association/home.html')

# df_query should be global
@csrf_exempt
def api_get_userkey_associate(request):
    userkey = request.POST.get('userkey')
    cate = request.POST['cate'] # This is an alternative way to get POST data.
    cond = request.POST.get('cond')
    weeks = int(request.POST.get('weeks'))
    key = userkey.split()

    #global  df_query # global variable It's not necessary.

    df_query = filter_dataFrame_fullText(key, cond, cate,weeks)

    # if df_query is empty, return an error message
    if len(df_query) == 0:
        return JsonResponse({'error': 'No results found for the given keywords.'})
    
    newslinks = get_title_link_topk(df_query, k=15)
    related_words, clouddata = get_related_word_clouddata(df_query)
    same_paragraph = get_same_para(df_query, key, cond, k=10) # multiple keywords


    response = {
        'newslinks': newslinks,
        'related_words': related_words,
        'same_paragraph': same_paragraph,
        'clouddata':clouddata,
        'num_articles': len(df_query),
    }
    return JsonResponse(response)

# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()

    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() -
                  timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date)

    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: news category
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all(
            (qk in text) for qk in user_keywords))  # 寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any(
            (qk in text) for qk in user_keywords))  # 寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# get titles and links from k pieces of news 
def get_title_link_topk(df_query, k=25):
    items = []
    for i in range( len(df_query[0:k]) ): # show only 10 news
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']
        photo_link = df_query.iloc[i]['photo_link']
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''
        
        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
            'photo_link': photo_link
        }

        items.append(item_info)
    return items 

# Get related keywords by counting the top keywords of each news.
# Notice:  do not name function as  "get_related_keys",
# because this name is used in Django
def get_related_word_clouddata(df_query):

    # wf_pairs = get_related_words(df_query)
    # prepare wf pairs 
    counter=Counter()
    for idx in range(len(df_query)):
        pair_dict = dict(eval(df_query.iloc[idx].top_key_freq))
        counter += Counter(pair_dict)
    wf_pairs = counter.most_common(20) #return list format

    # cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20
    textSizeMax = 120
    # Scaling frequency value into an interval of from 20 to 120.
    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 


# Step1: split paragraphs in text 先將文章切成一個段落一個段落
def cut_paragraph(text):
    paragraphs = text.split('。')  # 遇到句號就切開
    #paragraphs = re.split('。', text) # 遇到句號就切開
    #paragraphs = re.split('[。！!？?]', text) # 遇到句號(也納入問號、驚嘆號、分號等)就切開
    paragraphs = list(filter(None, paragraphs))
    return paragraphs

# Step2: Select all paragraphs where multiple keywords occur.


def get_same_para(df_query, user_keywords, cond, k=30):
    same_para = []
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            para += "。"
            if cond == 'and':
                if all([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
            elif cond == 'or':
                if any([re.search(kw, para) for kw in user_keywords]):
                    same_para.append(para)
    return same_para[0:k]


    
print("app_user_keyword_association was loaded!")


ModuleNotFoundError: No module named 'app_user_keyword'

# For reference